In [ ]:
import sys
import os
import subprocess
from multiprocessing import Pool
from multiprocessing import cpu_count
import math

In [ ]:
url = "./source.txt"

def list_urls(file):

    with open("./source.txt",'r') as fh:
        source_files_list = list(line.strip() for line in fh.readlines())
    return source_files_list
    
    
def download_dataset(url):
    try:
        subprocess.call(f"wget {url}",shell=True)
    except:
        pass

DATADIR = "/scratch1/rgoli/aws_customer_reviews"
def unzip_files(DATADIR):
    gz_files = os.listdir(DATADIR)
    source_files_list = list(line.strip() for line in gz_files)
    for file_ in source_files_list:
    #     print(file_)
        print(subprocess.call(["gzip","-d",os.path.join(DATADIR,file_)],shell=False))

In [ ]:
source_files_list = list(filter(lambda file_:file_.endswith(".tsv"),os.listdir(DATADIR)))

In [ ]:
import pyspark
print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_CONFIG_FILE'])
print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_MASTER_HOST'])
print(os.environ['SPARK_MASTER_PORT'])
print(os.environ['SPARK_MASTER_WEBUI_PORT'])

In [ ]:
import nltk
# nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words('english')
PUNCTUATIONS = """[!"#$%&()*+-.\/\\:;<=>?@\[\]^_`{|}\t\n\',~—]"""

In [ ]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark import SparkContext
from pyspark.sql.types import StructType,StructField, StringType
import gzip
sc = SparkContext()

In [ ]:
spark = pyspark.sql.SparkSession(sc)

In [ ]:
from timeit import default_timer as timer

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import CountVectorizer
from timeit import default_timer as timer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, ArrayType


import time
import pandas as pd

In [ ]:
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [ ]:
import csv
def read_gunzip(path):
#     with gzip.open(path,'rt') as fh:
    data = spark.read.csv(path,sep="\t")
    return data
def read_tsv(path):
#     print(path)
    data = spark.read.csv(path,sep="\t",header = True)
    data = data.select(["review_body"])
    return data

In [ ]:
data_files = list(filter(lambda x:x.endswith(".tsv"),os.listdir(DATADIR)))

In [ ]:
review_objects = [file_.split("_")[3] for file_ in data_files]
data_files_parallelized = sc.parallelize([os.path.join(DATADIR,file_) for file_ in data_files])

In [ ]:
data_files_parallelized_list = data_files_parallelized.collect()

In [ ]:
read_t1 = timer()
dataframes = list(map(lambda file_:read_tsv(file_),data_files_parallelized_list))
read_t2 = timer()
print("Read time: ",read_t2 - read_t1)

In [ ]:
size = 0
for file_ in data_files:
    size += round(os.path.getsize(os.path.join(DATADIR,file_))*9.31*1e-10,2)
    print(round(size,2),int(int(size) % 12))

## ALL Helper functions

In [ ]:

def tf_idf_pre_processing(dataframe,column_name):
    dataframe = dataframe.na.drop(subset=[column_name])
    dataframe = dataframe.withColumn(column_name,f.lower(f.col(column_name)))
    dataframe = dataframe.withColumn(column_name,f.regexp_replace(f.col(column_name), PUNCTUATIONS, ' '))
    dataframe = dataframe.withColumn(column_name,f.trim(f.col(column_name)))
    tokenizer = Tokenizer(inputCol=column_name, outputCol="words")
    dataframe = tokenizer.transform(dataframe)                  
    stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=STOPWORDS)
    data = stopwordsRemover.transform(dataframe)
    return data

def hash_tf(dataframe):
    starttime = timer()
#     tokenizer_hastingtf = Tokenizer(inputCol="review_body", outputCol="tokenized")
#     tokenized_data = tokenizer_hastingtf.transform(dataframe)
    hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
    result = hashingTF.transform(dataframe)
    print("Time to Hash", timer() - starttime)
    return result

def count_tf(dataframe):
    countVectors = CountVectorizer(inputCol="filtered", outputCol="rawFeatures", vocabSize=100000, minDF=5)
    model = countVectors.fit(dataframe)
    result = model.transform(dataframe)
    return result


def idf_generator(dataframe):
    starttime = timer()
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfmodel = idf.fit(dataframe)
    idf_data = idfmodel.transform(dataframe)
    print("Time to idf", timer() - starttime)
    return idf_data

# TF IDF with HasingTF and IDF

## Performance data size vs Computation Time

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
column_name = "review_body"
while True:
    size = 0
    time_dict = {}
    i=0
    for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        print(size)
        if i==0:
            i += 1
            continue
        if int(size) >= 60:
            starttime = timer()
            hashing_data = tf_idf_pre_processing(full_data_frame,column_name)
            result_hash = hash_tf(hashing_data)
            idf_data_hash = idf_generator(result_hash)
            time_to_idf = timer() - starttime
            time_dict.update({size:time_to_idf})
            print("Time to HashingTFIDF:", time_to_idf)
            break
            

In [ ]:
metric_folder = "./metrics_4node"
with open(os.path.join(metric_folder,f"perf_results_hashingtf_{12}.csv"),"a") as fh:
    fh.write("Size"+","+"Time"+"\n")
    for line in time_mapping:
        fh.write(f"{line[0]}",+",",f"line{1}")

## Performing time computation

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
size = 0
for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        print(size)
        full_data_frame = full_data_frame.union(df)
        if size >= 0.5:
            break

In [ ]:
column_name = "review_body"
starttime = timer()
hashing_data = tf_idf_pre_processing(full_data_frame,column_name)
result_hash = hash_tf(hashing_data)
idf_data_hash = idf_generator(result_hash)
time_to_idf = timer() - starttime
print("Time to HashingTFIDF:", timer() - starttime)

## TF IDF with CountVectorizer

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
column_name = "review_body"
while True:
    size = 0
    time_dict_count = {}
    i=0
    for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        if int(int(size) % 60) == 0:
            starttime = timer()
            data = tf_idf_pre_processing(full_data_frame,column_name)
            result = count_tf(data)
            idf_data = idf_generator(result)
            time_to_idf = timer() - starttime
            time_dict_count.update({size:time_to_idf})
            print("Time to CountingTFIDF:", time_to_idf)
    break
            

In [ ]:
metric_folder = "./metrics_4node"
with open(os.path.join(metric_folder,f"perf_results_counttf_{48}.csv"),"a") as fh:
    fh.write("Size"+","+"Time"+"\n")
    for line in sorted(time_dict_count.keys()):
        fh.write(f"{line}"+","+f"{time_dict_count[line]}"+"\n")

## Performing time computation

In [ ]:
emptyschema = StructType([
  StructField('review_body', StringType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
full_data_frame = spark.createDataFrame(emptyRDD,emptyschema)

In [ ]:
size = 0
for file_, df in list(zip(data_files_parallelized_list,dataframes)):
        size += round(os.path.getsize(file_)*9.31*1e-10,2)
        full_data_frame = full_data_frame.union(df)
        if size > 1.6:
            print(size)
            break

In [ ]:
column_name = "review_body"
starttime = timer()
data = tf_idf_pre_processing(full_data_frame,column_name)
result = count_tf(data)
idf_data = idf_generator(result)
time_to_idf = timer() - starttime
# time_dict_count.update({size:time_to_idf})
print("Time to CountingTFIDF:", time_to_idf)